In [2]:
from lxml import etree
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import numpy as np
from PIL import ImageDraw
from random import randint

In [2]:
def get_bbox(path):
    path = 'Annotations/' + path
    tree = etree.parse(path)
    root = tree.getroot()
    xmin,ymin,xmax,ymax = [], [], [], []
    for j in root.findall('object'):
        for i in j.findall('bndbox'):
            xmin.append(i.find('xmin').text)
            ymin.append(i.find('ymin').text)
            ymax.append(i.find('ymax').text)
            xmax.append(i.find('xmax').text)
    return xmin, ymin, xmax, ymax

In [180]:
print xmin[1], ymin[1]
print zero_arr[xmin[1],ymin[1]]
print np.argpartition(zero_arr, 0).shape

1 235
1.0
(500, 335)


/Users/Ramana/projects/macvnev/lib/python2.7/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


In [181]:
def floodfill(matrix, x, y):
    #"hidden" stop clause - not reinvoking for "c" or "b", only for "a".
    if matrix[x][y] == 0:  
        matrix[x][y] = 2 
        #recursively invoke flood fill on all surrounding cells:
        if x > 0:
            floodfill(matrix,x-1,y)
        if x < len(matrix[y]) - 1:
            floodfill(matrix,x+1,y)
        if y > 0:
            floodfill(matrix,x,y-1)
        if y < len(matrix) - 1:
            floodfill(matrix,x,y+1)

In [3]:
def crop_image(xmin, ymin, xmax, ymax, j):
    image_path = 'JPEGImages/' +j
    original = Image.open(image_path)
    for i in range(0, len(xmin)):
        original.crop((int(xmin[i]), int(ymin[i]), int(xmax[i]), int(ymax[i]))).save("Cropped/"+ str(i) + "-" +str(j))

In [4]:
test_path = 'JPEGImages/2007_000042.jpg'
original = Image.open(test_path)
original.crop((0, 100, 450, 300)).show()

In [5]:
#Add all the XML files to the list
onlyfiles = []
path = 'Annotations/'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

In [121]:
k = 1
for j in onlyfiles:
    xmin, ymin, xmax, ymax = get_bbox(j)
    crop_image(xmin, ymin, xmax, ymax, j.split('.')[0]+".jpg")
    k += 1

ValueError: invalid literal for int() with base 10: '45.70000076293945'

In [127]:
#Testing Block
for k in range(14737, len(onlyfiles)-1):
    xmin, ymin, xmax, ymax = get_bbox(onlyfiles[k])
    crop_image(xmin, ymin, xmax, ymax, onlyfiles[k].split('.')[0]+".jpg")
    k += 1

In [191]:
bin_rand = np.random.randint(0,2,(40,40))


[[1 0]
 [0 1]]


In [6]:
for j in range(1453, len(onlyfiles)):
    xmin, xmax, ymin, ymax = get_bbox(onlyfiles[j])
    img_path = onlyfiles[j].split('.')[0]+ ".jpg"
    original = Image.open('JPEGImages/' + img_path)
    zero_arr = np.zeros(original.size)
    for i in range(len(xmin)):
        zero_arr[int(ymin[i]):int(ymax[i]), int(xmin[i]):int(xmax[i])] = 1
    xmin1,ymin1,xmax1,ymax1 = [],[],[],[]
    for i in range(zero_arr.shape[0]-32):
        for k in range(zero_arr.shape[1]-32):
            if np.count_nonzero(zero_arr[i:i+32, k:k+32])==0:
                xmin1.append(k)
                ymin1.append(i)
                xmax1.append(k+32)
                ymax1.append(i+32)
            if len(xmin1) > 4:
                break
    n_back = 5
    if len(xmin1) < 5:
        n_back = len(xmin1)
    for ntemp in range(0,n_back):
        n_img = original.crop((xmin1[ntemp], ymin1[ntemp], xmax1[ntemp], ymax1[ntemp]))
        n_img.save('Background/' +str(ntemp)+ "-" +img_path)

ValueError: invalid literal for int() with base 10: '45.70000076293945'

In [200]:
a = np.random.randint(0,3,(4,4))
print a

[[2 0 0 0]
 [0 0 2 1]
 [2 1 0 2]
 [0 0 2 1]]


In [250]:
a[1:3, 0:4]

array([[0, 0, 2, 1],
       [2, 1, 0, 2]])

In [3]:
#Add jpg files to list
back_files, obj_files = [], []
path1 = 'Background/'
back_files = [f for f in listdir(path1) if isfile(join(path1, f))]
path2 = 'Cropped/'
cropped_files = [f for f in listdir(path2) if isfile(join(path2, f))]

In [6]:
with open('train1.txt', 'w+') as f2:
    for i in range(1, len(cropped_files)):
        f2.write(path1+cropped_files[i]+ " " +str(1))
        f2.write("\n")
    for j in range(1, len(back_files)):
        f2.write(path2+back_files[j] + " " +str(0))
        f2.write("\n")

In [285]:
print len(back_files)

1453


In [12]:
import random
with open('train1.txt','r') as source:
    data = [ (random.random(), line) for line in source ]
data.sort()
with open('train4.txt','w+') as target:
    for _, line in data:
        target.write( line )

In [12]:
for kj in cropped_files[1:]:
    b_file = Image.open(path2+kj)
    b_file = b_file.resize((32, 32))
    b_file.save(path2+kj)

In [8]:
f6 = open('val.txt', 'w+')
count_temp = 0
for l in open('train4.txt', 'r'):
    count_temp += 1
    if count_temp<4000:
        continue
    else:
        f6.write(l)
    if count_temp == 6000:
        break

    

In [13]:
f5 = open('train.txt', 'w+')
count_temp = 0
for l in open('train4.txt', 'r'):
    f5.write(l)
    if count_temp == 4000:
        break
    count_temp += 1